This notebook does the following:

    generate random walk polymer chain
    local optimization using openbabel
    packing of multiple polymer chains into the simulation box
    Compute the system topology and read in the force field data
    write the data file for lammps run
    write the main input file for lammps run

Dependencies:

    pymatgen(latest development version from github or atleast v3.7.1)

Required input files:

    structures of start, bulk and terminal monomer molecules
    appropriately formatted yaml forcefield data file

In [1]:
import numpy as np

from pprint import pprint

from pymatgen import Molecule
from pymatgen.io.babel import BabelMolAdaptor
from pymatgen.io.lammps.utils import Polymer, PackmolRunner

# Set the start, bulk, and terminal molecules. 

Set charges and optionally the forcefield species name mapping 

In [2]:
# start molecule
peo_start = Molecule.from_file("test_files/peo_start.xyz")
s_charges = [-0.1187, 0.0861, 0.0861, 0.0861, -0.2792, -0.0326, 0.0861, 0.0861]
peo_start.add_site_property("charge", s_charges)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("test_files/peo_bulk.xyz")
b_charges = [-0.0326, 0.0861, 0.0861, -0.2792, -0.0326, 0.0861, 0.0861]
peo_bulk.add_site_property("charge", b_charges)
head = 0
tail = 4

# terminal molecule
peo_end = Molecule.from_file("test_files/peo_end.xyz")
e_charges = [-0.0326, 0.0861, 0.0861, -0.2792, -0.1187, 0.0861, 0.0861, 0.0861]
peo_end.add_site_property("charge", e_charges)
e_head = 0
e_tail = 4

# Create polymer via random walk

In [3]:
n_units = 58
link_distance = 1.5075

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

# linear chain
peo_polymer_linear = Polymer(peo_start, s_head, s_tail, 
                             peo_bulk, head, tail, 
                             peo_end, e_head, e_tail, 
                             n_units, link_distance, linear_chain=True)

# Set the topology of the polymer from the linear chain

In [4]:
from pymatgen.io.lammps.topology import Topology

topology = Topology.from_molecule(peo_polymer_linear.molecule)

natoms, nbonds, nangles, ndih =  len(topology.atoms), len(topology.bonds), len(topology.angles), len(topology.dihedrals)

print(natoms, nbonds, nangles, ndih)

(408, 407, 754, 861)


# Generate polymer matrix

In [5]:
# constituent molecules
molecules = [peo_polymer.molecule]
#Polymer matrix settings
matrix_config = [{"number": 1, "inside box":[0,0,0,100,100,100]}]

# Use Packmol to pack the polymer chains into a box

In [6]:
pmr = PackmolRunner(molecules,
                    matrix_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    output_file="poly_packed.xyz")
packed_polymer = pmr.run()

packed molecule written to poly_packed.xyz


# Read in the force field parameters

In [7]:
from pymatgen.io.lammps.force_field import ForceField

forcefield = ForceField.from_file("ff_data.yaml")

# Generate lammps data file from the topology and the forcefield

In [8]:
from pymatgen.io.lammps.data import LammpsForceFieldData

mols_number = [mol_config["number"] for mol_config in matrix_config]
box_size = [[0.0, 100], 
            [0.0, 100], 
            [0.0, 100]]

# list of molecular topologies for each constituent molecules
topologies = [topology]*len(molecules)

lammps_ff_data = LammpsForceFieldData.from_forcefield_and_topology(molecules, mols_number, 
                                                                   box_size, packed_polymer, 
                                                                   forcefield, topologies)
#write the data file
#lammps_ff_data.write_data_file("lammps_data.dat")

# Setup input file

In [9]:
from pymatgen.io.lammps.input import DictLammpsInput

#override default settings read from the json file with these
user_settings = {"pair_style": "buck/coul/cut 15.0", 
                 "pair_coeff": ["1 1 2649.6 0.2674 27.22",
                                "1 2 4320.0 0.2928 137.6",
                                "1 3 14176.0 0.2563 104.0",
                                "2 2 14976.0 0.3236 637.6",
                                "2 3 33702.4 0.2796 503.0",
                                "3 3 75844.8 0.2461 396.9"],
                  "dump": ["DUMP all custom 200 poly.lammps.dump id type x y z ix iy iz mol",
                           "traj all dcd 100 lammps_traj.dcd"]}

lammps_input = DictLammpsInput.from_file("NPT", "NPT.json", 
                                         lammps_data=lammps_ff_data, data_filename="peo_chain.data", 
                                         user_lammps_settings=user_settings)
lammps_input.write_input("lammps.in")
print str(lammps_input)

Writing the data to peo_chain.data
units  real
dimension  3
atom_style  full
boundary  p p p
neighbor  10.0 bin
bond_style  harmonic
angle_style  harmonic
dihedral_style  opls
read_data  peo_chain.data
pair_style  buck/coul/cut 15.0
pair_coeff 1 1 2649.6 0.2674 27.22
pair_coeff 1 2 4320.0 0.2928 137.6
pair_coeff 1 3 14176.0 0.2563 104.0
pair_coeff 2 2 14976.0 0.3236 637.6
pair_coeff 2 3 33702.4 0.2796 503.0
pair_coeff 3 3 75844.8 0.2461 396.9
dump DUMP all custom 200 poly.lammps.dump id type x y z ix iy iz mol
dump traj all dcd 100 lammps_traj.dcd
thermo_style  custom step temp press etotal fmax fnorm
thermo  100
fix  NPT all npt temp 298 298 100.0 iso 0.0 0.0 100.0
run  1000



# Run Lammps

In [11]:
# binary compiled with openmpi, from ubuntu repo
#!lammps-daily -in lammps.in

# serial binary, compiled from source
#!lmp_serial -in lammps.in

#
#!mpiexec -n 4 lammps -in lammps.in

LAMMPS (18 Jun 2016)
Reading data file ...
  orthogonal box = (0 0 0) to (100 100 100)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  408 atoms
  scanning bonds ...
  4 = max bonds/atom
  scanning angles ...
  6 = max angles/atom
  scanning dihedrals ...
  9 = max dihedrals/atom
  reading bonds ...
  407 bonds
  reading angles ...
  754 angles
  reading dihedrals ...
  861 dihedrals
Finding 1-2 1-3 1-4 neighbors ...
 Special bond factors lj:   0          0          0         
 Special bond factors coul: 0          0          0         
  4 = max # of 1-2 neighbors
  6 = max # of 1-3 neighbors
  12 = max # of 1-4 neighbors
  14 = max # of special neighbors
Neighbor list info ...
  1 neighbor list requests
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 25
  ghost atom cutoff = 25
  binsize = 12.5, bins = 8 8 8
Setting up Verlet run ...
  Unit style    : real
  Current step  : 0
  Time step     : 1
Me